In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C, Datasets as D, Utilities as U, Parsers as P
from mppi.Models import TwoLevelSystems as TLS
from mppi.Utilities import Constants as Const
import matplotlib.pyplot as plt
import numpy as np
import os, yaml

h_red = Const.Planck_reduced_ev_ps*1e3 # hbar in eV*fs
h = Const.Planck_ev_ps*1e3 # h in eV*fs

In [3]:
omp = 1
mpi = 4

In [4]:
code = C.YamboCalculator(omp=omp,mpi=mpi,executable='yambo_rt')

Initialize a Yambo calculator with scheduler direct


# Computation of the IP Absorption spectrum

We compute the IP absorption spectrum using Yambo

In [8]:
# analysis with k sampling volume of 0.06 (1000 points)     
run_dir = 'ABS-IP_k1000_kspan0.06'
source_dir = 'NSCF/bnds_8-kpoints_1000-sampling_random_close_Gamma_kspan0.06.save'

In [21]:
U.build_SAVE(source_dir,run_dir,make_link=True)

Create folder ABS-IP_k1000_kspan0.06
Executing command: cd NSCF/bnds_8-kpoints_1000-sampling_random_close_Gamma_kspan0.06.save; p2y -a 2
Create a symlink of /home/marco/Data/RICERCA/DFT AND MANY BODY/OPTICAL ORIENTATION/Echo mechanisms and decay time/PHOTON_ECO/NSCF/bnds_8-kpoints_1000-sampling_random_close_Gamma_kspan0.06.save/SAVE in ABS-IP_k1000_kspan0.06
Executing command: cd ABS-IP_k1000_kspan0.06;OMP_NUM_THREADS=1 yambo


In [12]:
h/250

0.016542670784

In [14]:
T2 = 250 # damping time in fs
eta = h / T2 # eta damping in eV 

inp = I.YamboInput('yambo -o c',folder=run_dir)
inp['variables']['LongDrXd'] = [[1.00,1.00,1.00],''] # electric field
inp['variables']['XfnQP_E'] = [[0.95477,1.00,1.00],''] # scissor
inp['variables']['DmRngeXd'] = [[eta/2,eta/2],'eV'] # damping range
inp['variables']['EnRngeXd'] = [[0,6],'eV'] # energy range
inp['variables']['BndsRnXd'] = [[2,8],''] # bands used for transitions
inp['variables']['QpntsRXd'] = [[1,1],''] # momentum transfer
inp

{'args': 'yambo -o c',
 'folder': 'ABS-IP_k1000_kspan0.06',
 'filename': 'yambo.in',
 'arguments': ['chi', 'optics'],
 'variables': {'X_Threads': [0.0, ''],
  'DIP_Threads': [0.0, ''],
  'ETStpsXd': [100.0, ''],
  'Chimod': 'IP',
  'QpntsRXd': [[1, 1], ''],
  'BndsRnXd': [[2, 8], ''],
  'EnRngeXd': [[0, 6], 'eV'],
  'DmRngeXd': [[0.008271335392, 0.008271335392], 'eV'],
  'LongDrXd': [[1.0, 1.0, 1.0], ''],
  'XfnQP_E': [[0.95477, 1.0, 1.0], '']}}

In [19]:
name = 'ip_abs'
result = code.run(input=inp,name=name,run_dir=run_dir,skip=False)
result

delete folder: ABS-IP_k1000_kspan0.06/ip_abs
run command: mpirun -np 4 yambo_rt -F ip_abs.in -J ip_abs -C ip_abs
computation ip_abs is running...
computation ip_abs ended

        There are no o-* files.
        Maybe you have performed a ypp computation or wait_end_run and/or
        the dry_run option are active?
        Otherwise a possible error has occured during the computation
        
Game_over string not found in report. Check the computation!


{'output': [],
 'report': 'ABS-IP_k1000_kspan0.06/ip_abs/r-ip_abs_optics_chi',
 'dft': 'ABS-IP_k1000_kspan0.06/SAVE/ns.db1'}

In [ ]:
data = P.YamboParser(result['output'][0])

In [ ]:
eps = data['eps_q1_ip']
energy = eps['col0']
eps_im = eps['col1']
eps_re = eps['col2']

In [ ]:
plt.plot(energy,eps_im)